In [29]:
import os
import torch
import torchaudio
import numpy as np
from tensorflow.keras.models import load_model
from scipy.io.wavfile import write
from IPython.display import Audio, display

# Directories and model paths
model_dir = "D:/NEU/NUwork/Voice Cloning Web Application Project/VoiceCloner/media/models"
tacotron2_path = os.path.join(model_dir, "tacotron2_statedict.pt")
waveglow_path = os.path.join(model_dir, "waveglow_256channels_universal_v5.pt")

# Verify that the files exist
assert os.path.exists(tacotron2_path), f"Tacotron2 model not found at {tacotron2_path}"
assert os.path.exists(waveglow_path), f"WaveGlow model not found at {waveglow_path}"


In [30]:
def load_waveglow_model(model_path):
    model = torch.load(model_path)['model']
    model = model.remove_weightnorm(model)
    model.eval()
    return model

# Load WaveGlow model
waveglow = load_waveglow_model(waveglow_path)


C:\Users\10195\AppData\Local\Temp\ipykernel_18556\4246807772.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)['model']
d:\NEU\NUwork\Voice 

In [31]:
def load_waveglow_model(model_path):
    model = torch.load(model_path)['model']
    model = model.remove_weightnorm(model)
    model.eval()
    return model

# Load WaveGlow model
waveglow = load_waveglow_model(waveglow_path)


C:\Users\10195\AppData\Local\Temp\ipykernel_18556\4246807772.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)['model']


In [32]:
def text_to_sequence(text, max_len=500, n_mels=80):
    sequence = [ord(char) for char in text]
    if len(sequence) > max_len:
        sequence = sequence[:max_len]
    else:
        sequence = sequence + [0] * (max_len - len(sequence))
    sequence = np.array(sequence)
    sequence = sequence.reshape((1, max_len))
    sequence_3d = np.zeros((1, n_mels, max_len))
    sequence_3d[0, 0, :] = sequence
    return sequence_3d

def generate_mel_spectrogram(text, model_path):
    model = load_model(model_path)
    sequence = text_to_sequence(text)
    mel_spec = model.predict(sequence)
    return mel_spec


In [33]:
def mel_spectrogram_to_audio(mel_spec, waveglow):
    mel_spec = torch.tensor(mel_spec).unsqueeze(0).to('cuda')
    with torch.no_grad():
        audio = waveglow.infer(mel_spec)
    return audio.cpu().numpy()

def generate_audio(text, model_path, waveglow_path):
    mel_spec = generate_mel_spectrogram(text, model_path)
    waveglow = load_waveglow_model(waveglow_path)
    audio = mel_spectrogram_to_audio(mel_spec[0], waveglow)
    return audio


In [34]:
# Paths
model_path = "D:/NEU/NUwork/Voice Cloning Web Application Project/VoiceCloner/media/models/test_tts_model.keras"
result_path = "D:/NEU/NUwork/Voice Cloning Web Application Project/VoiceCloner/media/synthesized/test_synthesized_audio.wav"

# Generate audio
text = "Hello, this is a test."
audio = generate_audio(text, model_path, waveglow_path)
write(result_path, 22050, audio)
print(f"Generated audio saved to {result_path}")

# Display audio player
display(Audio(audio, rate=22050))


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 253952, but received input with shape (1, 158720)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 80, 500), dtype=float32)
  • training=False
  • mask=None